In [14]:
from google.colab import drive

drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
#Import Packages
import pandas as pd
import numpy as np
from scipy.stats import uniform # for training-and-test split
import statsmodels.api as sm # statistical models (including regression)
import statsmodels.formula.api as smf # R-like model specification
import matplotlib.pyplot as plt # 2D plotting
import seaborn as sns # PROVIDES TRELLIS AND SMALL MULTIPLE PLOTTING
from pathlib import Path # for merging dataframes
from sklearn.preprocessing import LabelEncoder #to numberize columns
from sklearn.preprocessing import OneHotEncoder #another way to numberize object columns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#import required packages
import re
from pprint import pprint
import nltk; nltk.download('stopwords')
import sklearn
import datetime
from datetime import datetime, timedelta
from pandas import DataFrame
import sys
! {sys.executable} -m pip install progress #progress bars
from progress.bar import Bar

pd.set_option("display.max_colwidth", 200)

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy for lemmatization
import sys
! {sys.executable} -m pip install spacy
! {sys.executable} -m pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz
import spacy 


#from spacy_langdetect import LanguageDetector
#nlp = spacy.load('en')
#nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

# Plotting tools
! {sys.executable} -m pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

### A_DATA

In [16]:
#COMPANY A
#read in data

a_data_raw = pd.read_csv('/content/gdrive/Shareddrives/IST 718 Project/a_data_BoolOut.csv') #with boolean indicating outlier
a_data_raw

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,assignment_group,number,state,priority,short_description,description,comments_and_work_notes,work_notes,close_notes,location,sys_created_on,closed_at,u_initial_response_sla_completed,bus_dur_num,cal_dur_num,reassignment_count,sys_mod_count,assigned_to,closed_by,contact_type,sys_created_by,sla_due,u_task_number_flag,sys_class_name,sys_updated_on,sys_updated_by,urgency,work_start,opened_at,active,comments,closed_by.average_daily_fte,closed_by.city,activity_due,approval,location.city,impact,knowledge,u_reason_to_hold,location.country,location.u_location_id,location.state,location.u_region,u_actual_category,u_actual_subcategory,u_cause,u_cause_code,u_qs_fcresolve,u_misrouted,due_date,sys_domain,task_effective_number,TicketDuration,CalendarDuration,BusinessDuration,category,subcategory,outlier
0,0,0,0,0,I-ACM-SD L1-India,INC1499259,Cancelled,P4,user need to install jet stream viewer,NaN,10-27-2020 13:29:00 - Rajat Kumar (Additional comments)\nIncident is Cancelled and a Request Item RITM0974470 is created.\n\n,NaN,NaN,belfast,2020-10-27 12:50:00,2020-10-27 13:29:00,False,2337,2337,1,1,Rajat Kumar,NaN,Chat,NaN,UNKNOWN,NaN,Incident,2020-10-27 13:29:00,7.0,2,10/27/2020 13:29,2020-10-27 12:50:00,False,10-27-2020 13:29:00 - rajat kumar (additional comments)\nincident is cancelled and a request item ritm0974470 is created.\n\n,NaN,los angeles,UNKNOWN,Not Yet Requested,belfast,3 - Low,False,NaN,united kingdom,GB-GNGBP681-0002/BLDG,county antrim,europe(emea),software,software breakfix,NaN,NaN,False,False,NaN,NaN,NaN,0 days 00:39:00,0:38:57,0:38:57,software,software breakfix,False
1,1,3,3,3,I-ACM-CTS-India,INC1498224,Cancelled,P4,need assistance turning on bitlocker,name:sanjay kumar\r\nphone:566539297\r\nemail address:sanjay.kumar4@aecom.com\r\nthe user called in stating his bitlocker is switched off on his computer and needs to switch it on \r\ntook remote ...,10-27-2020 12:22:04 - Kaveri S (Additional comments)\nIncident is Cancelled and a Request Item RITM0974445 is created.\n\n10-27-2020 12:21:54 - Kaveri S (Work notes)\nmcafee is installed on the c...,10-27-2020 12:21:54 - Kaveri S (Work notes)\nmcafee is installed on the computer\r\nbit locker is not enabled\r\ncreating a request ticket to remove mcafee\r\n\r\nKB0033325\n\n10-27-2020 12:20:37 ...,NaN,"riyadh, SA",2020-10-27 00:29:00,2020-10-27 12:22:00,False,1324,42739,1,12,Kaveri S,NaN,Phone,9.0,UNKNOWN,NaN,Incident,2020-10-27 12:22:00,9.0,2,10/27/2020 0:33,2020-10-27 00:29:00,False,10-27-2020 12:22:04 - kaveri s (additional comments)\nincident is cancelled and a request item ritm0974445 is created.\n\n10-27-2020 00:29:45 - abdul baig (additional comments)\nname:sanjay kumar...,NaN,los angeles,UNKNOWN,Not Yet Requested,riyadh,3 - Low,False,NaN,saudi arabia,NaN,NaN,middle east (emea),security,bitlocker - break/fix,NaN,NaN,False,False,NaN,NaN,NaN,0 days 11:53:00,11:52:19,0:22:04,security,bitlocker - break/fix,False
2,2,4,4,4,I-ACM-SD Self Serv,INC1498051,Cancelled,P4,unable to obtain autocad license at home,description: i am trying to obtain a license for autocad at home. it pops up with an error message saying one cannot be borrowed this time and to contact my system administrator.,10-27-2020 02:58:38 - Sakshi Srivastava (Additional comments)\nIncident is Cancelled and a Request Item RITM0973929 is created.\n\n,NaN,NaN,perth,2020-10-26 20:12:00,2020-10-27 02:58:00,False,0,24359,0,2,Sakshi Srivastava,NaN,Self-service,1.0,UNKNOWN,NaN,Incident,2020-10-27 02:58:00,9.0,3,10/26/2020 20:30,2020-10-26 20:12:00,False,10-27-2020 02:58:38 - sakshi srivastava (additional comments)\nincident is cancelled and a request item ritm0973929 is created.\n\n,NaN,los angeles,UNKNOWN,Not Yet Requested,perth,3 - Low,False,NaN,australia,NaN,western australia,australia new zealand (apac),software,software breakfix,NaN,NaN,False,False,NaN,NaN,NaN,0 days 06:46:00,6:45:59,0:00:00,software,software breakfix,False
3,3,6,6,6,I-ACM-CTS-India,INC1497739,Cancell

In [17]:
keep_columns = ['number', 'opened_at', 'category', 'subcategory', 'assignment_group', 'priority', 'sys_mod_count', 
                'contact_type', 'bus_dur_num', 'assigned_to', 'reassignment_count', 'outlier'] #use these columns for analysis

a_data = a_data_raw[keep_columns]
a_data['opened_at'] = pd.to_datetime(a_data.opened_at)
a_data['Date'] = a_data['opened_at'].dt.strftime('%Y-%m-%d')
a_data = a_data.sort_values('Date')
a_data.reset_index(inplace=True)

In [18]:
#Subset Data to begin 04/01/2020
a_data[a_data['Date']=='2020-04-01']
a_brief = a_data[36227:]
a_brief = a_brief[a_brief['outlier'] == False]
a_brief

,index,number,opened_at,category,subcategory,assignment_group,priority,sys_mod_count,contact_type,bus_dur_num,assigned_to,reassignment_count,outlier,Date
36227,62806,INC1240385,2020-04-01 13:02:00,software,software breakfix,I-ACM-CTS-India,P3,11,Phone,25077,Anushka Shetty,0,False,2020-04-01
36228,62805,INC1240387,2020-04-01 13:04:00,software,software breakfix,I-ACM-SD Self Serv,P4,30,Self-service,111305,Manohar Oberoi,0,False,2020-04-01
36229,62804,INC1240390,2020-04-01 13:06:00,software,software breakfix,I-ACM-DS-NA-US-NORTHWEST,P3,2,NaN,18115,Keith Beers,0,False,2020-04-01
36230,62819,INC1240327,2020-04-01 12:28:00,email,email-break / fix,I-ACM-CTS-India,P4,5,Phone,27095,Sanchaita Biswas,0,False,2020-04-01
36231,62801,INC1240397,2020-04-01 13:13:00,access-issue/request,vpn 2fa - anyconnect how to,I-ACM-SD L1-India,P4,3,Phone,871,Akash M,0,False,2020-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99322,4,INC1497518,2020-10-26 11:08:00,software,software breakfix,I-ACM-SD L1-India,P4,1,Chat,0,Manohar Oberoi,0,False,2020-10-26
99323,3,INC1497739,2020-10-26 13:34:00,workstation/ computer - software,windows 10,I-ACM-CTS-India,P4,1,Phone,4216,Sanchaita Biswas,0,False,2020-10-26
99324,2,INC1498051,2020-10-26 20:12:00,software,software breakfix,I-ACM-SD Self Serv,P4,2,Self-service,0,Sakshi Srivastava,0,False,2020-10-26
99325,1,INC1498224,2020-10-27 00:29:00,security,bitlocker - break/fix,I-ACM-CTS-India,P4,12,Phone,1324,Kaveri S,1,False,2020-10-27


In [26]:
print(np.unique(a_brief['reassignment_count']))
print(np.unique(a_brief['sys_mod_count']))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 23 24]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 65 66 67 68 69 70 71 72
 73 74 76 77 78 80 82 83 84 85 87 88 91 92 93 98 99]


In [20]:
a_brief.columns
#overall count - DONE
#assignment groups make count for each? - DONE
#sys_mod_count take avg
#contact_Type take count of each? - DONE
#bus_dur_num take avg - DONE
#reassignment_count take avg - DONE
#outlier take count - DONE

Index(['index', 'number', 'opened_at', 'category', 'subcategory',
       'assignment_group', 'priority', 'sys_mod_count', 'contact_type',
       'bus_dur_num', 'assigned_to', 'reassignment_count', 'outlier', 'Date'],
      dtype='object')

In [21]:
np.unique(a_brief['contact_type'].astype(str))

array(['Autoticketing AT&T', 'Autoticketing Dynamic Automation',
       'Autoticketing NetCool', 'Autoticketing Orion', 'Chat', 'Email',
       'Phone', 'Self-service', 'Service Catalog', 'Walk-in', 'Walk-up',
       'nan'], dtype=object)

In [30]:
assign_groups = ['I-ACM-CTS-India', 'I-ACM-DS-NA-US-CC-NY-EXECSUPP',
       'I-ACM-DS-NA-US-NORTHEAST', 'I-ACM-DS-NA-US-NORTHWEST',
       'I-ACM-DS-NA-US-SOUTHEAST', 'I-ACM-DS-NA-US-San Fran-EXECSUPP',
       'I-ACM-SD L1-India', 'I-ACM-SD Self Serv']
contact_types = ['Autoticketing AT&T', 'Autoticketing Dynamic Automation',
       'Autoticketing NetCool', 'Autoticketing Orion', 'Chat', 'Email',
       'Phone', 'Self-service', 'Service Catalog', 'Walk-in', 'Walk-up'] #drop nan
categories = ['access-issue/request', 'software', 'application-issue/request', 'general', 'email', 'computer hardware']

#WRITE FUNCTION FOR CONVERTING DF CATEGORIES INTO TIMESERIES DATAFRAMES
def convert_cat_for_timeseries_a(source_df, category):

  dates = np.unique(source_df['Date'])
  
  newDF = pd.DataFrame()
  newDF['date'] = dates
  newDF['date'] = pd.to_datetime(newDF['date'], format='%Y-%m-%d')

  #'access-issue/request'
  catDF = source_df[source_df['category']==category]

  daily_count = [] #empty list for agg daily ticket count
  busdur_avg = [] #empty list for avg business duration (minutes)
  smc_avg = [] #empty list for avg system mod counts
  reassignc_avg = [] #empty list for avg rassignment counts
  outlier_count = [] #empty list for count of gross outliers in day
  adj_bdn = [] #adjusted business duration (bus_dur_num/sys_mod_count)
  #assignment group counts
  CTS_India = []
  NY_Exec = []
  US_NE = []
  US_NW = []
  US_SE = []
  US_SanFran = []
  L1_India = []
  Self_Serve = []
  #contact type counts
  ATT = []
  Dyn_Auto = []
  NetCool = []
  Orion = []
  Chat = []
  Email = []
  Phone = []
  Self_Serve_ct = []
  Serv_Cat = []
  Walk_In = []
  Walk_Up = []

  for date in dates:

    df = catDF[catDF['Date']==date]

    #ticket count
    a = len(df)
    daily_count.append(a)

    #outlier count
    true_count = df['outlier'].sum()
    outlier_count.append(true_count)

    #average business duration (minutes)
    df['bus_dur_num'].fillna(df['bus_dur_num'].mean(), inplace=True)
    bdn = round(df['bus_dur_num'].sum()/len(df),2)
    busdur_avg.append(bdn)

    #average count of system_mod_count
    df['sys_mod_count'].fillna(df['sys_mod_count'].mean(), inplace=True)
    df['sys_mod_count'] = df['sys_mod_count'] + 1
    smc = round(df['sys_mod_count'].sum()/len(df),2)
    smc_avg.append(smc)

    #average count of 'reassignment_count'
    df['reassignment_count'].fillna(df['reassignment_count'].mean(), inplace=True)
    rac = round(df['reassignment_count'].sum()/len(df),2)
    reassignc_avg.append(rac)

    #adjusted business duraction
    adj = bdn/smc
    adj_bdn.append(adj)

    #assignment group counts
    df_cts = df[df['assignment_group']==assign_groups[0]]
    cts = len(df_cts)
    CTS_India.append(cts)
    df_nye = df[df['assignment_group']==assign_groups[1]]
    nye = len(df_cts)
    NY_Exec.append(nye)
    df_usne = df[df['assignment_group']==assign_groups[2]]
    usne = len(df_usne)
    US_NE.append(usne)
    df_usnw = df[df['assignment_group']==assign_groups[3]]
    usnw = len(df_usnw)
    US_NW.append(usnw)
    df_usse = df[df['assignment_group']==assign_groups[4]]
    usse = len(df_usse)
    US_SE.append(usse)
    df_ussf = df[df['assignment_group']==assign_groups[5]]
    ussf = len(df_ussf)
    US_SanFran.append(ussf)
    df_l1 = df[df['assignment_group']==assign_groups[6]]
    l1 = len(df_l1)
    L1_India.append(l1)
    df_ss = df[df['assignment_group']==assign_groups[7]]
    ss = len(df_ss)
    Self_Serve.append(ss)

    #contact_type counts
    df_att = df[df['contact_type']==contact_types[0]]
    att = len(df_att)
    ATT.append(att)
    df_da = df[df['contact_type']==contact_types[1]]
    da = len(df_da)
    Dyn_Auto.append(da)
    df_nc = df[df['contact_type']==contact_types[2]]
    nc = len(df_nc)
    NetCool.append(nc)
    df_o = df[df['contact_type']==contact_types[3]]
    o = len(df_o)
    Orion.append(o)
    df_ch = df[df['contact_type']==contact_types[4]]
    ch = len(df_ch)
    Chat.append(ch)
    df_e = df[df['contact_type']==contact_types[5]]
    e = len(df_e)
    Email.append(e)
    df_ph = df[df['contact_type']==contact_types[6]]
    ph = len(df_ph)
    Phone.append(ph)
    df_ssct = df[df['contact_type']==contact_types[7]]
    ssct = len(df_ssct)
    Self_Serve_ct.append(ssct)
    df_sc = df[df['contact_type']==contact_types[8]]
    sc = len(df_sc)
    Serv_Cat.append(sc)
    df_wi = df[df['contact_type']==contact_types[9]]
    wi = len(df_wi)
    Walk_In.append(wi)
    df_wu = df[df['contact_type']==contact_types[10]]
    wu = len(df_wu)
    Walk_Up.append(wu)

  #ADD TO DATAFRAME
  newDF['count'] = daily_count
  newDF['outlier_count'] = outlier_count
  newDF['bdn_avg'] = busdur_avg
  newDF['sys_mod_count_avg'] = smc_avg
  newDF['reassign_count_avg'] = reassignc_avg
  newDF['adj_bdn'] = adj_bdn
  #assignment_groups
  newDF['ag_cts_india_count'] = CTS_India
  newDF['ag_ny_exec_count'] = NY_Exec
  newDF['ag_us_ne_count'] = US_NE
  newDF['ag_us_nw_count'] = US_NW
  newDF['ag_us_se_count'] = US_SE
  newDF['ag_us_sanfran_count'] = US_SanFran
  newDF['ag_l1_india_count'] = L1_India
  newDF['ag_self_serve_count'] = Self_Serve
  #contact_type
  newDF['ct_auto_att_count'] = ATT
  newDF['ct_auto_dynamic_count'] = Dyn_Auto
  newDF['ct_auto_netcool_count'] = NetCool
  newDF['ct_auto_orion_count'] =Orion
  newDF['ct_chat_count'] = Chat
  newDF['ct_email_count'] = Email
  newDF['ct_phone_count'] = Phone
  newDF['ct_self_serve_count'] = Self_Serve_ct
  newDF['ct_service_catalog_count'] = Serv_Cat
  newDF['ct_walk_in_count'] = Walk_In
  newDF['ct_walk_up_count'] = Walk_Up
  return newDF


In [31]:
pd.options.mode.chained_assignment = None  # default='warn'
categories = ['access-issue/request', 'software', 'application-issue/request', 
              'general', 'email', 'computer hardware']

#Call Function to generate DFs
access_df = convert_cat_for_timeseries_a(a_brief, 'access-issue/request')
software_df = convert_cat_for_timeseries_a(a_brief, 'software')
application_df = convert_cat_for_timeseries_a(a_brief, 'application-issue/request')
general_df = convert_cat_for_timeseries_a(a_brief, 'general')
email_df = convert_cat_for_timeseries_a(a_brief, 'email')
hardware_df = convert_cat_for_timeseries_a(a_brief, 'computer hardware')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:64: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:75: RuntimeWarning: invalid value encountered in long_scalars


In [32]:
hardware_df

,date,count,outlier_count,bdn_avg,sys_mod_count_avg,reassign_count_avg,adj_bdn,ag_cts_india_count,ag_ny_exec_count,ag_us_ne_count,ag_us_nw_count,ag_us_se_count,ag_us_sanfran_count,ag_l1_india_count,ag_self_serve_count,ct_auto_att_count,ct_auto_dynamic_count,ct_auto_netcool_count,ct_auto_orion_count,ct_chat_count,ct_email_count,ct_phone_count,ct_self_serve_count,ct_service_catalog_count,ct_walk_in_count,ct_walk_up_count
0,2020-04-01,17,0,143280.59,17.88,0.65,8013.455817,0,0,3,4,7,0,2,0,0,0,0,0,1,1,7,7,0,0,1
1,2020-04-02,16,0,79751.31,10.94,0.38,7289.882084,1,1,3,7,2,0,3,0,0,0,0,0,0,3,5,5,0,1,1
2,2020-04-03,14,0,72982.93,11.43,0.21,6385.208224,0,0,3,4,5,0,1,0,0,0,0,0,4,0,4,4,0,0,0
3,2020-04-04,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2020-04-05,1,0,28800.00,10.00,0.00,2880.000000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2020-10-21,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
204,2020-10-22,3,0,86751.00,5.00,0.00,17350.200000,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,3,0,0,0
205,2020-10-23,0,0,NaN,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
206,2020-10-26,1,0,28800.00,7.00,1.00,4114.285714,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### J_DATA

In [35]:
#COMPANY J
#read in data

j_data_raw = pd.read_csv('/content/gdrive/Shareddrives/IST 718 Project/j_data_BoolOut.csv') #with boolean indicating outlier
j_data_raw['category'] = j_data_raw['category'].replace(np.nan, 'Task') #give tasks category 'task'
j_data_raw['subcategory'] = j_data_raw['subcategory'].replace(np.nan, 'Task') #give tasks subcategory 'task'
j_data_raw

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,8,10,11,13,15,18,19,20,25,29,30,31,33,35,36,38,39,40,42,43,45,53,54,55,56,57,58,59,60,65,67,68,69,70,72,73,74,75,76,77,78,86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Unnamed: 0.1,number,opened_at,incident_state,sys_updated_by,category,short_description,caller_id.location,assignment_group,u_time_last_workgroup_first_se,u_resolution_rejected_report,u_resolution_rejected,u_rejected_from_assignedto,u_reject_count,u_last_transfer_time,u_last_incident_state,u_last_assignment_group,u_initial_support_group,u_initial_priority,u_incident_rejected,urgency,sys_mod_count,sys_updated_on,time_worked,u_time_resolved,sys_class_name,u_task_state,state,severity,priority,u_pending_reminder_sent,u_pending_user_last_workgroup_,u_operational_task,u_notify_caller_update,notify,u_new_requirement,made_sla,u_last_service_call_update,u_last_priority,...,sys_domain,sla_due,sys_created_by,sys_created_on,contact_type,u_contact_windows_team,u_contact_unix_team,u_contact_unified_messaging_te,u_contact_network_team,u_contact_foundation_team,u_contact_dba_team,u_contact_client_technology_gr,u_close_code,bus_dur_num,activity_due,active,assigned_to,subcategory,reassignment_count,expected_start,parent,due_date,closed_by,closed_at,work_end,request_item,request,TicketDuration,ExpectedTaskDuration,TaskDuration,CloseWorkDiff,TotalSlaDuration,ULastWorkgroupDuration,TimeFirstTransfer,PendingUserDuration,Pending3rdPartyDuration,CalendarDuration,BusinessDuration,last_service_call_update_datetime,outlier
0,0,0,INC3148753,2020-06-09 20:46:00,Closed,system,Desktop or laptop,\tI have opened laptop refresh for more than 1 month and I have not received any updates.This is the ticket number REQ599157,"Sunnyvale, CA",ONSITE-AMERICAS,2020-06-10 18:45:00,Accepted Resolution,0.0,pthakur,0.0,2020-06-10 15:45:00,6.0,onsite-americas,itco,P3,False,2,6,2020-06-22 03:02:00,0,2020-06-11 17:26:00,Incident,Closed,Closed Complete,3 - Low,P3,False,81659.0,False,Yes,Do not notify,False,True,"Jun 22, 2020 00:02:12 system: Incident automatically closed after 10 days in the Resolved state.",P3,...,global,2020-06-24 05:03:00,neelamvijay,2020-06-09 20:46:00,Self-service,False,False,False,False,False,False,False,Resolved,713708,6/15/2020 18:45,False,ttruong,other,1,NaN,NaN,NaN,NaN,2020-06-22 03:02:00,NaN,NaN,NaN,1 days 20:40:00,NaN,NaN,NaN,2:59:54,2:59:54,18:58:54,22:40:59,0:00:00,"12 days, 6:15:17","8 days, 6:15:08",2020-06-22 00:02:12,False
1,1,1,INC2960955,2020-02-04 07:46:00,Closed,system,Easy connect,I can't seem to connect via Pulse anymore with my MAC. This was working just fine yesterday. It is giving me invalid certificate. Are there some instructions I can follow?,"Westford, MA",ONSITE-AMERICAS,2020-02-04 09:55:00,Accepted Resolution,0.0,talmquist,0.0,2020-02-04 07:47:00,6.0,onsite-americas,onsite-americas,P3,False,2,4,2020-02-15 03:03:00,0,2020-02-04 14:18:00,Incident,Closed,Closed Complete,3 - Low,P3,False,0.0,False,Yes,Do not notify,False,True,"Feb 15, 2020 00:03:26 system: Incident automatically closed after 10 days in the Resolved state.",P3,...,global,2020-02-19 13:28:00,talmquist,2020-02-04 07:47:00,Incident Form,False,False,False,False,False,False,False,Resolved,735188,UNKNOWN,False,ccoffee,report an issue,0,NaN,NaN,NaN,NaN,2020-02-15 03:03:00,NaN,NaN,NaN,0 days 06:32:00,NaN,NaN,NaN,6:30:39,2:07:36,0:00:00,0:00:00,0:00:00,"10 days, 19:16:42","8 days, 12:13:08",2020-02-15 00:03:26,False
2,2,2,INC3131802,2020-05-27 11:06:00,Closed,system,Desktop or laptop,""" i"" key not working : virtual keyboard works fine","Houston, TX",ONSITE-AMERICAS,2020-05-27 12:43:00,Accepted Resolution,0.0,pthakur,0.0,2020-05-27 12:27:00,4.0,onsite-americas,onsite-americas,P3,False,2,6,2020-06-07 03:02:00,2,2020-06-07 03:02:00,Incident,Closed,Closed Complete,3 - Low,P3,True,915534.0,False,Yes,Do not notify,False,True,"Jun 07, 2020 00:02:12 system: Incident automatically closed after 10 days in the Pending (User) state.",P3,...,global,2020-06-10 18:23:00,pthakur,2020-05-27 12:27:00,Incident Form,False,False,False,False,False,False,False,Cancelled,636783,6/6/2020 12:43,False,tdefranco,desktop / laptop hardware issue,0,NaN,NaN,NaN,NaN,2020-06-07 03:02:00,NaN,NaN,

In [36]:
keep_columns = ['number', 'opened_at', 'category', 'subcategory', 'assignment_group', 'priority', 'sys_mod_count', 
                'contact_type', 'bus_dur_num', 'assigned_to', 'reassignment_count', 'outlier'] #use these columns for analysis

j_data = j_data_raw[keep_columns]
j_data['opened_at'] = pd.to_datetime(j_data.opened_at)
j_data['Date'] = j_data['opened_at'].dt.strftime('%Y-%m-%d')
j_data = j_data.sort_values('Date')
j_data.reset_index(drop=True)

,number,opened_at,category,subcategory,assignment_group,priority,sys_mod_count,contact_type,bus_dur_num,assigned_to,reassignment_count,outlier,Date
0,TASK463646,2018-10-12 01:41:00,Task,Task,ONSITE-AMERICAS,NaN,8,Incident Form,30305537,tholzhey,0,False,2018-10-12
1,TASK498961,2019-06-10 13:28:00,Task,Task,ONSITE-AMERICAS,NaN,8,Incident Form,11168951,cvu,0,False,2019-06-10
2,TASK499151,2019-06-11 15:53:00,Task,Task,ONSITE-AMERICAS,NaN,9,Incident Form,11653408,cvu,0,False,2019-06-11
3,TASK500072,2019-06-17 17:38:00,Task,Task,ONSITE-AMERICAS,NaN,3,Incident Form,11412790,gchen,0,False,2019-06-17
4,TASK502052,2019-06-28 16:15:00,Task,Task,ONSITE-AMERICAS,NaN,9,Incident Form,9949351,cvu,0,False,2019-06-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17094,TASK568116,2020-10-24 08:00:00,Task,Task,ONSITE-AMERICAS,NaN,7,Incident Form,58197,cvu,0,False,2020-10-24
17095,TASK568147,2020-10-25 02:01:00,Task,Task,ONSITE-AMERICAS,NaN,7,Incident Form,58453,cvu,0,False,2020-10-25
17096,TASK568154,2020-10-25 02:01:00,Task,Task,ONSITE-AMERICAS,NaN,7,Incident Form,56758,cvu,0,False,2020-10-25
17097,TASK568226,2020-10-26 11:26:00,Task,Task,ONSITE-AMERICAS,NaN,2,Incident Form,9633,aangeles,0,False,2020-10-26


In [38]:
#Subset Data to begin 04/01/2020
j_data[j_data['Date']=='2020-04-01']
j_brief = j_data[7780:]
j_brief.reset_index(drop=True)
j_brief = j_brief[j_brief['outlier'] == False]
j_brief


,number,opened_at,category,subcategory,assignment_group,priority,sys_mod_count,contact_type,bus_dur_num,assigned_to,reassignment_count,outlier,Date
7653,INC3044564,2020-04-01 12:04:00,Desktop or laptop,other,ONSITE-AMERICAS,P4,5,Incident Form,633336,ttruong,0,False,2020-04-01
15201,TASK543755,2020-04-01 17:13:00,Task,Task,ONSITE-AMERICAS,NaN,9,Incident Form,5686986,howardng,0,False,2020-04-01
13159,TASK543779,2020-04-01 21:43:00,Task,Task,ONSITE-AMERICAS,NaN,8,Incident Form,501015,howardng,0,False,2020-04-01
7815,INC3044335,2020-04-01 08:53:00,Desktop or laptop,desktop / laptop application issue,ONSITE-AMERICAS,P3,5,Self-service,644773,ccoffee,1,False,2020-04-01
13298,TASK543713,2020-04-01 11:52:00,Task,Task,ONSITE-AMERICAS,NaN,4,Incident Form,769660,talmquist,0,False,2020-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17041,TASK568116,2020-10-24 08:00:00,Task,Task,ONSITE-AMERICAS,NaN,7,Incident Form,58197,cvu,0,False,2020-10-24
17039,TASK568147,2020-10-25 02:01:00,Task,Task,ONSITE-AMERICAS,NaN,7,Incident Form,58453,cvu,0,False,2020-10-25
17049,TASK568154,2020-10-25 02:01:00,Task,Task,ONSITE-AMERICAS,NaN,7,Incident Form,56758,cvu,0,False,2020-10-25
17063,TASK568226,2020-10-26 11:26:00,Task,Task,ONSITE-AMERICAS,NaN,2,Incident Form,9633,aangeles,0,False,2020-10-26


In [61]:
print(len(a_brief))
print(len(j_brief))

categories_a = ['access-issue/request', 'software', 'application-issue/request', 'general', 'email', 'computer hardware']

categories_j = ['Task','Desktop or laptop', 'Access and passwords', 'Application support', 'Infrastructure services', 'Easy connect', 'Telephone & Conference (Audio, Video)']

print(len(a_brief[a_brief['category'].isin(categories_a)]))
print(len(j_brief[j_brief['category'].isin(categories_j)]))

62926
9151
56417
8315


In [87]:
a_data.reset_index(drop=True)
a_data[a_data['Date']=='2020-05-31']
a_1 = a_data[4391:53435]
a_1ac = a_1[a_1['category']=='access-issue/request']
a_1['bus_dur_num'].mean()

78629.12291003997

In [77]:
np.unique(j_brief['category'])

array(['Access and passwords', 'Application support',
       'Collaboration and content tools', 'Desktop or laptop',
       'Easy connect', 'Email / Calendar', 'Engineering',
       'ININ (Contact Center service)', 'Infrastructure Monitoring',
       'Infrastructure services', 'Microsoft Skype services',
       'Mobility devices and services', 'Printers and peripherals',
       'Software', 'Task', 'Telephone & Conference (Audio, Video)'],
      dtype=object)

In [40]:
j_brief.columns
#overall count - DONE
#assignment groups make count for each? - DONE
#sys_mod_count take avg
#contact_Type take count of each? - DONE
#bus_dur_num take avg - DONE
#reassignment_count take avg - DONE
#outlier take count - DONE

Index(['number', 'opened_at', 'category', 'subcategory', 'assignment_group',
       'priority', 'sys_mod_count', 'contact_type', 'bus_dur_num',
       'assigned_to', 'reassignment_count', 'outlier', 'Date'],
      dtype='object')

In [41]:
print(np.unique(j_brief['assignment_group'].astype(str)))
print(np.unique(j_brief['contact_type'].astype(str)))
print(np.unique(j_brief['category'].astype(str)))

['IT-INFRA-CTG' 'ONSITE-AMERICAS']
['CM-self-service' 'Incident Form' 'Phone' 'Self-service']
['Access and passwords' 'Application support'
 'Collaboration and content tools' 'Desktop or laptop' 'Easy connect'
 'Email / Calendar' 'Engineering' 'ININ (Contact Center service)'
 'Infrastructure Monitoring' 'Infrastructure services'
 'Microsoft Skype services' 'Mobility devices and services'
 'Printers and peripherals' 'Software' 'Task'
 'Telephone & Conference (Audio, Video)']


In [42]:
assign_groups = ['IT-INFRA-CTG', 'ONSITE-AMERICAS']
contact_types = ['CM-self-service', 'Incident Form', 'Phone', 'Self-service']
categories = ['Task','Desktop or laptop', 'Access and passwords', 'Application support', 'Infrastructure services', 'Easy connect', 'Telephone & Conference (Audio, Video)']


#WRITE FUNCTION FOR CONVERTING DF CATEGORIES INTO TIMESERIES DATAFRAMES
def convert_cat_for_timeseries_j(source_df, category):

  dates = np.unique(source_df['Date'])
  
  newDF = pd.DataFrame()
  newDF['date'] = dates
  newDF['date'] = pd.to_datetime(newDF['date'], format='%Y-%m-%d')

  #'access-issue/request'
  catDF = source_df[source_df['category']==category]

  daily_count = [] #empty list for agg daily ticket count
  busdur_avg = [] #empty list for avg business duration (minutes)
  smc_avg = [] #empty list for avg system mod counts
  reassignc_avg = [] #empty list for avg rassignment counts
  outlier_count = [] #empty list for count of gross outliers in day
  adj_bdn = [] #adjusted business duration (bus_dur_num/sys_mod_count)
  #assignment group counts
  Infra_CTG = []
  USA = []
  #contact type counts
  CM_Self_Serve = []
  Inc_Form = []
  Phone = []
  Self_Serve = []

  for date in dates:

    df = catDF[catDF['Date']==date]

    #ticket count
    a = len(df)
    daily_count.append(a)

    #outlier count
    true_count = df['outlier'].sum()
    outlier_count.append(true_count)

    #average business duration (minutes)
    df['bus_dur_num'].fillna(df['bus_dur_num'].mean(), inplace=True)
    bdn = round(df['bus_dur_num'].sum()/len(df),2)
    busdur_avg.append(bdn)

    #average count of system_mod_count
    df['sys_mod_count'].fillna(df['sys_mod_count'].mean(), inplace=True)
    df['sys_mod_count'] = df['sys_mod_count'] + 1
    smc = round(df['sys_mod_count'].sum()/len(df),2)
    smc_avg.append(smc)

    #average count of 'reassignment_count'
    df['reassignment_count'].fillna(df['reassignment_count'].mean(), inplace=True)
    rac = round(df['reassignment_count'].sum()/len(df),2)
    reassignc_avg.append(rac)

    #adjusted business duraction
    adj = bdn/smc
    adj_bdn.append(adj)

    #assignment group counts
    df_cts = df[df['assignment_group']==assign_groups[0]]
    cts = len(df_cts)
    Infra_CTG.append(cts)
    df_nye = df[df['assignment_group']==assign_groups[1]]
    nye = len(df_cts)
    USA.append(nye)

    #contact_type counts
    df_att = df[df['contact_type']==contact_types[0]]
    att = len(df_att)
    CM_Self_Serve.append(att)
    df_da = df[df['contact_type']==contact_types[1]]
    da = len(df_da)
    Inc_Form.append(da)
    df_nc = df[df['contact_type']==contact_types[2]]
    nc = len(df_nc)
    Phone.append(nc)
    df_o = df[df['contact_type']==contact_types[3]]
    o = len(df_o)
    Self_Serve.append(o)

  #ADD TO DATAFRAME
  newDF['count'] = daily_count
  newDF['outlier_count'] = outlier_count
  newDF['bdn_avg'] = busdur_avg
  newDF['sys_mod_count_avg'] = smc_avg
  newDF['reassign_count_avg'] = reassignc_avg
  newDF['adj_bdn'] = adj_bdn
  #assignment_groups
  newDF['ag_infra_ctg_count'] = Infra_CTG
  newDF['ag_onsite_americas_count'] = USA
  #contact_type
  newDF['ct_cm_self_service_count'] = CM_Self_Serve
  newDF['ct_incident_form_count'] = Inc_Form
  newDF['ct_phone_count'] = Phone
  newDF['ct_self_service_count'] = Self_Serve
  return newDF


In [48]:
pd.options.mode.chained_assignment = None  # default='warn'
categories = ['Desktop or laptop', 'Access and passwords', 
              'Application support', 'Infrastructure services', 
              'Easy connect', 'Telephone & Conference (Audio, Video)']


#Call Function to generate DFs
task_df = convert_cat_for_timeseries_j(j_brief, 'Task')
desktop_laptop_df = convert_cat_for_timeseries_j(j_brief, 'Desktop or laptop')
access_pw_df = convert_cat_for_timeseries_j(j_brief, 'Access and passwords')
application_support_df = convert_cat_for_timeseries_j(j_brief, 'Application support')
infrastructure_df = convert_cat_for_timeseries_j(j_brief, 'Infrastructure services')
easy_connect_df = convert_cat_for_timeseries_j(j_brief, 'Easy connect')
tele_conference_df = convert_cat_for_timeseries_j(j_brief, 'Telephone & Conference (Audio, Video)')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:53: RuntimeWarning: invalid value encountered in long_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in long_scalars


In [53]:
task_df['count'].sum() + desktop_laptop_df['count'].sum() + access_pw_df['count'].sum() + application_support_df['count'].sum() + infrastructure_df['count'].sum() + easy_connect_df['count'].sum() + tele_conference_df['count'].sum()

8315

# Outputting TimeSeries DFs

In [34]:
#A Data
#EXPORT FILES TO DRIVE
access_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/a_ts_access.csv')
software_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/a_ts_software.csv')
application_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/a_ts_application.csv')
general_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/a_ts_general.csv')
email_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/a_ts_email.csv')
hardware_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/a_ts_hardware.csv')

In [54]:
#J Data
#EXPORT FILES TO DRIVE
task_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_task.csv')
desktop_laptop_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_desktop.csv')
access_pw_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_access.csv')
application_support_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_application.csv')
infrastructure_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_infrastructure.csv')
easy_connect_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_connect.csv')
tele_conference_df.to_csv('/content/gdrive/Shared drives/IST 718 Project/j_ts_teleconference.csv')